In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.optimizers import Adam,SGD
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
from keras.regularizers import l1,l2
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score
import tensorflow as tf
import os
import random as rn
import numpy as np
import logging
import time
from util_functions import *

# disable GPUs for test reproducibility
tf.config.set_visible_devices([], 'GPU')

SEED=0

DATASET_FOLDER = "./DOS2019_Binary"

X_train, y_train = load_dataset(DATASET_FOLDER + "/*" + '-train.hdf5')
X_val, y_val = load_dataset(DATASET_FOLDER + "/*" + '-val.hdf5')
X_test, y_test = load_dataset(DATASET_FOLDER + "/*" + '-test.hdf5')

In [ ]:
def compileModel(model,optimizer='sgd', lr=0.001):
    if optimizer == 'sgd':
        optimizer = SGD(learning_rate=lr, momentum=0.0)
    else:
        optimizer = Adam(learning_rate=lr, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
    model.compile(loss='binary_crossentropy', optimizer=optimizer,metrics=['accuracy']) 

In [ ]:
# Function to create the MLP model (for GridSearchCV)
def create_model(optimizer='sgd', dense_layers=4, hidden_units=2, learning_rate = 0.001, dropout_rate=0, activation='relu'):
    model = Sequential(name  = "mlp")
    model.add(Dense(hidden_units, input_shape=(21,), activation='relu'))
    model.add(Dropout(dropout_rate))
    for layer in range(dense_layers):
        model.add(Dense(hidden_units, activation='relu', name='hidden-fc' + str(layer)))
        model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation='sigmoid', name='fc2'))
    compileModel(model, optimizer,learning_rate)
    return model

In [ ]:
# Create a KerasClassifier based on the create_model function
model = KerasClassifier(build_fn=create_model, batch_size=100, verbose=1)

PATIENCE = 10
k=2 # number of folds for cross-validation

# Define the hyperparameters to tune and their possible values
param_grid = {
    'learning_rate' : [0.001,0.01],
    'optimizer' : ['sgd','adam'],
    'hidden_units' : [16,32],
    'dropout_rate': [0,0.9]
}

# Perform grid search with 5-fold cross-validation
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=k)

### Add early stopping
early_stopping = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=PATIENCE, restore_best_weights=True)
start_time = time.time()
grid_result = grid.fit(X_train, y_train, epochs=100, validation_data=(X_val, y_val), callbacks= [early_stopping])
stop_time = time.time()

# Total training time
print("Total training time (sec): ", stop_time-start_time)
# Print the best parameters and corresponding accuracy
print("Best parameters found: ", grid_result.best_params_)
print("Best cross-validated accuracy: {:.2f}".format(grid_result.best_score_))

# Evaluate the best model on the test set
best_model = grid.best_estimator_
test_accuracy = best_model.score(X_test, y_test)
print("Test accuracy of the best model: {:.2f}".format(test_accuracy))

# Save the best model   
best_model.model.save('best_model.h5')